In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import logging
import sys
import time, datetime
from typing import Dict, Iterable
import os
import gzip
import json

from google.colab import drive

# Mount Google Drive to the current Colab session
drive.mount('/content/drive')

# Verify that your drive is accessible
!ls /content/drive/MyDrive

Mounted at /content/drive
'16 б.gdoc'			  deleteme.zip		   share
'6 фото.     5 лет блондин.JPG'   futuristic-office.jpeg  'Tanya Passport.JPG'
'Colab Notebooks'		  rental_houses.gsheet	  'лечение травами 3.gdoc'


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset

device  = 'cuda'
dataset = load_dataset("openai_humaneval")

README.md:   0%|          | 0.00/6.52k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/83.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

In [ ]:
def remove_triple_backticks_at_edges(text: str) -> str:
    """
    Removes triple backticks (` ``` `) if they appear at the beginning or the end of the text.

    :param text: The input text as a string.
    :return: A string with triple backticks removed from the edges.
    """
    s1 = "```python\n"
    s2 = "```python"
    s3 = "```"
    for s in [s1, s2, s3]:
        if text.startswith(s):
            text = text[len(s):].lstrip()      # Remove a version of backticks + extra whitespace at the start
    if text.endswith(s3):
        text = text[: -len(s3)].rstrip()       # Remove the backticks + extra whitespace at the end
    return text


def add_import(text: str) -> str:
    ''' Adds import statements stripped by LLM '''
    text = text.strip()
    s = 'from typing import List'
    if s not in text:
        text = s + '\n\n' + text
    return text

In [ ]:
def write_jsonl(filename: str, data: Iterable[Dict], append: bool = False):
    """
    Writes an iterable of dictionaries to jsonl
    """
    if append:
        mode = 'ab'
    else:
        mode = 'wb'
    filename = os.path.expanduser(filename)
    if filename.endswith(".gz"):
        with open(filename, mode) as fp:
            with gzip.GzipFile(fileobj=fp, mode='wb') as gzfp:
                for x in data:
                    gzfp.write((json.dumps(x) + "\n").encode('utf-8'))
    else:
        with open(filename, mode) as fp:
            for x in data:
                fp.write((json.dumps(x) + "\n").encode('utf-8'))

In [2]:
model_path="m-a-p/OpenCodeInterpreter-DS-6.7B"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model     = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model.eval()
print(f'Model device: {model.device}')

tokenizer_config.json:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

Model device: cuda:0


In [3]:
print(f'Model temperature: {model.config.temperature}. Model top_p: {model.config.top_p}')

Model temperature: 1.0. Model top_p: 1.0


In [ ]:
def generate_response(prompt):
  inputs = tokenizer.apply_chat_template(
        [{'role': 'user', 'content': prompt }],
        return_tensors="pt"
  ).to(model.device)
  outputs = model.generate(
      inputs,
      max_new_tokens=1024,
      do_sample=False,
      pad_token_id=tokenizer.eos_token_id,
      eos_token_id=tokenizer.eos_token_id,
  )
  res = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
  return res

In [ ]:
# to save results
model_nickname = 'opencodeinterpreter-DS-6.7b'
time_stamp     = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-2]
file_name      = f'/content/drive/My Drive/Colab Notebooks/logs/{model_nickname}_samples_{time_stamp}.jsonl'

# prompt
prompt_prefix = '''
1. Complete the following starter code as code that can be run directly by a Python interpreter.
2. Without modifying the starter code, concatenate it with the completion.
3. Output only the code and no irrelevant language. Do not say "completion:" or "Here is the completed function" or "Here is a completion" or anything like that. Do not use tripple backticks or any other irrelevant symbols. The output must be only runnable Python code.
4. Do not provide any example usage or any other code after the function. Output only the function and nothing after it.

Starter code:\n
'''

In [ ]:
num_samples_per_task = 1
completions = []

for item in dataset["test"]:
    print(item['task_id'])
    #if item['task_id'] == 'HumanEval/5':
    #    break
    for i in range(num_samples_per_task):
        start_time  = time.time()
        full_prompt = prompt_prefix + item["prompt"]
        #full_prompt = task_body["prompt"]
        try:
            completion  = generate_response( full_prompt )
        except Exception as e:
            completion = f"Error when model predicted a completion:\n{e}"

        completion = remove_triple_backticks_at_edges(completion.strip())
        completions.append( {'task_id': item['task_id'], 'completion': completion} )
        write_jsonl(file_name, completions)
        print('Full prompt:\n', full_prompt, '\n', '='*75, '\n', sep='')
        print('Completion:\n', completion, sep='')
        print(f"\nTime elapsed: {(time.time() - start_time):.4f} seconds\n", '\n', '='*75, '\n', sep='')

Streaming output truncated to the last 5000 lines.
4. Do not provide any example usage or any other code after the function. Output only the function and nothing after it.

Starter code:


def circular_shift(x, shift):
    """Circular shift the digits of the integer x, shift the digits right by shift
    and return the result as a string.
    If shift > number of digits, return digits reversed.
    >>> circular_shift(12, 1)
    "21"
    >>> circular_shift(12, 2)
    "12"
    """


Completion:
def circular_shift(x, shift):
    digits = str(x)
    if shift > len(digits):
        return digits[::-1]
    else:
        return digits[shift:] + digits[:shift]

Time elapsed: 1.9628 seconds


HumanEval/66
Full prompt:

1. Complete the following starter code as code that can be run directly by a Python interpreter.
2. Without modifying the starter code, concatenate it with the completion.
3. Output only the code and no irrelevant language. Do not say "completion:" or "Here is the completed funct